In [22]:
import requests
import os
import urllib.parse
import posixpath
import pandas as pd
import paramiko
from traitlets.config import Config
from traitlets.config.application import Application

In [ ]:
# All the settings for the assignment (will become script arguments)
assignment = 'worksheet_01'
due_day = 'sat' # should be sat or wed
grader = 'timberst' # should be graders cwl

In [23]:
# make course an object
class course():
    
    def __init__(self, canvasHostName, courseID):
        self.canvasHostName = canvasHostName
        self.courseID = courseID
        print('Create class with Canvas host name:{0} and course ID: {1})'.
              format(self.canvasHostName, self.courseID))
        
    def get_student_ids(self):
        '''Read Canvas authentication token from an environment variable, takes a
        Canvas host name (includes https://) and the Canvas course id and returns
        a list of the student id's of all students currently enrolled in the course.
    
        Example:
        course.get_student_ids()'''
        #canvas_token = rudaux2_config.CANVAS_TOKEN
        canvas_token = os.environ["CANVAS_TOKEN"]
        url_path = posixpath.join("api", "v1", "courses", self.courseID, "enrollments")
        api_url = urllib.parse.urljoin(self.canvasHostName, url_path)
        resp = requests.get(
              url = api_url,
              headers = {
                "Authorization": f"Bearer {canvas_token}",
                "Accept": "application/json+canvas-string-ids"
              },
              json={
                "enrollment_type": ["student"],
                "per_page": "500"
              },
            )
        students = resp.json()
        student_id = []
        for student in students:
            student_id.append(student['user_id'])
        return student_id
    
    
    def get_student_info(self):
        '''Read Canvas authentication token from an environment variable, takes a
        Canvas host name (includes https://) and the Canvas course id and returns
        a list of the student id's, given and surnames of all students currently 
        enrolled in the course.
    
        Example:
        course.get_student_info()'''
        canvas_token = os.environ["CANVAS_TOKEN"]
        url_path = posixpath.join("api", "v1", "courses", self.courseID, "enrollments")
        api_url = urllib.parse.urljoin(self.canvasHostName, url_path)
        resp = requests.get(
              url = api_url,
              headers = {
                "Authorization": f"Bearer {canvas_token}",
                "Accept": "application/json+canvas-string-ids"
              },
              json={
                "enrollment_type": ["student"],
                "per_page": "500"
              },
            )
        students = resp.json()
        return students
    
    def get_assignments(self):
        '''Read Canvas authentication token from an environment variable, takes a
        Canvas host name (includes https://) and the Canvas course id and returns
        a Pandas data frame with all existing assignments and their attributes/data

        Example:
        course.get_assignments()'''
        #canvas_token = rudaux2_config.CANVAS_TOKEN
        canvas_token = os.environ["CANVAS_TOKEN"]
        url_path = posixpath.join("api", "v1", "courses", self.courseID, "assignments")
        api_url = urllib.parse.urljoin(self.canvasHostName, url_path)
        resp = requests.get(
          url=api_url,
          headers={
            "Authorization": f"Bearer {canvas_token}",
            "Accept": "application/json+canvas-string-ids"
          },
          json={
            "per_page": "2000"
          },
        )
        assignments = resp.json()
        assign_data = pd.DataFrame.from_dict(assignments)
        return assign_data
    
    def get_assignment_due_date(self, assignment):
        '''Takes the name of a Canvas assignment and returns the due date.
        
        Example:
        course.get_assignment_due_date('worksheet_01')'''
        assignments = self.get_assignments()
        assignment = assignments[['name', 'due_at']].query('name == @assignment')
        due_date = assignment['due_at'].to_numpy()[0]
        due_date = due_date.replace("T", "-")
        due_date = due_date.replace(":", "-")
        return due_date[:16]

In [26]:
# All the settings for the course - so go in a config file
dsci100_canvasHostName = "https://canvas.ubc.ca"
dsci100_courseID = "40616"
course_storage_path = '/tank/home/dsci100'
stu_repo_name = 'dsci-100'
ins_repo_name = 'dsci-100-instructor'
assignment_release_path = 'materials'
snapshot_prefix = 'zfs-auto-snap_'
snapshot_delay = '10'

In [28]:
dsci100 = course(dsci100_canvasHostName, dsci100_courseID) # get these values 
students = dsci100.get_student_info()

Create class with Canvas host name:https://canvas.ubc.ca and course ID: 40616)


In [29]:
students

[{'id': '1682992',
  'user_id': '313452',
  'course_id': '40616',
  'type': 'StudentEnrollment',
  'created_at': '2019-06-21T00:35:19Z',
  'updated_at': '2019-09-03T06:02:03Z',
  'associated_user_id': None,
  'start_at': None,
  'end_at': None,
  'course_section_id': '44261',
  'root_account_id': '1',
  'limit_privileges_to_course_section': False,
  'enrollment_state': 'active',
  'role': 'StudentEnrollment',
  'role_id': '3',
  'last_activity_at': '2019-09-12T01:30:58Z',
  'last_attended_at': None,
  'total_activity_time': 1331,
  'grades': {'html_url': 'https://canvas.ubc.ca/courses/40616/grades/313452',
   'current_grade': None,
   'current_score': None,
   'final_grade': None,
   'final_score': 0.0,
   'unposted_current_score': None,
   'unposted_current_grade': None,
   'unposted_final_score': 0.0,
   'unposted_final_grade': None},
  'sis_account_id': 'SIS.UBC.DSCI',
  'sis_course_id': 'cbbee518-6eb4-4387-b35f-d06ea8362a26',
  'course_integration_id': 'ID:ctprod01.lms.it.ubc.ca-37

In [ ]:
# Code to run
dsci100 = course(canvasHostName, courseID) # get these values 
students = dsci100.get_student_ids()
due_date = dsci100.get_assignment_due_date(assignment)[:13]

# Create copy from and to paths
copy_from_path = os.path.join('.zfs', 'snapshot', snapshot_prefix + due_day + '-' + due_date + snapshot_delay)
copy_to_path = os.path.join(course_storage_path, grader, ins_repo_name, 'submitted')

In [ ]:
ssh = paramiko.SSHClient() 
ssh.load_host_keys(os.path.expanduser(os.path.join("~", ".ssh", "known_hosts")))
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect("hub-prod-dsci.stat.ubc.ca", username="stty2u")
sftp = ssh.open_sftp()

In [ ]:
#student_path_remote = os.path.join(course_storage_path, str(students[0]))

In [ ]:
#assignment_path = os.path.join(student_path_remote, copy_from_path, stu_repo_name, assignment_release_path, assignment, assignment + '.ipynb')

In [ ]:
#student_path_local = os.path.join(copy_to_path, str(students[0]))

In [ ]:
#submission_path = os.path.join(student_path_local, assignment, assignment + '.ipynb')

In [ ]:
#if not os.path.exists(student_path_local):
#    os.mkdir(student_path_local)
#    os.mkdir(os.path.join(student_path_local, assignment))  
#else:   
#    if not os.path.exists(os.path.join(student_path_local, assignment)):
#        os.mkdir(os.path.join(student_path_local, assignment))

In [ ]:
#sftp.get(remotepath=assignment_path, localpath=submission_path)

In [ ]:
# looping over student id
for student in students:
    student_path_remote = os.path.join(course_storage_path, str(student))
    assignment_path = os.path.join(student_path_remote, copy_from_path, stu_repo_name, assignment_release_path, assignment, assignment + '.ipynb')
    student_path_local = os.path.join(copy_to_path, str(student))
    submission_path = os.path.join(student_path_local, assignment, assignment + '.ipynb')
    
    if not os.path.exists(student_path_local):
        os.mkdir(student_path_local)
        os.mkdir(os.path.join(student_path_local, assignment))  
    else:   
        if not os.path.exists(os.path.join(student_path_local, assignment)):
            os.mkdir(os.path.join(student_path_local, assignment))
    try:
        sftp.get(remotepath=assignment_path, localpath=submission_path)
    except:
      pass

In [ ]:
sftp.close()
ssh.close()